# Gene ID mapping in Python with `MyGene`

**Author**: Clarence Mah (ckmah@ucsd.edu

Source material: https://docs.mygene.info/projects/mygene-py/en/latest/

ID mapping is a very common, often not fun, task for every bioinformatician. Supposedly you have a list of gene symbols or reporter ids from an upstream analysis, and then your next analysis requires to use gene ids (e.g. Entrez gene ids or Ensembl gene ids). So you want to convert that list of gene symbols or reporter ids to corresponding gene ids.

### Some relatable images...
<html>
    <img alt="gene excel meme" src="img/gene_excel_meme.jpeg" width=300>
    <img src="img/genes-excel-theverge.png", alt="Microsoft Excel renames genes, article from The Verge" width=350 style="border: 1px solid black">
</html>

Here we want to show you how to use **mygene** module in Python to do ID mapping quickly and easily. **mygene** is essentially a convenient Python module to access [MyGene.info](http://mygene.info) gene query web services.

Fun fact: **MyGene** was developed here in San Diego from the labs of [Dr. Andrew Su](https://sulab.org) and [Dr. Chunlei Wu](https://wulab.io) at The Scripps Research Institute.

## Import and instantiate MyGeneInfo class

In [ ]:
import mygene

mg = mygene.MyGeneInfo()

## Mapping gene symbols to Entrez gene ids

Suppose **xli** is a list of gene symbols you want to convert to entrez gene ids:

In [ ]:
xli = [
    "CCDC83",
    "MAST3",
    "FLOT1",
    "RPL11",
    "ZDHHC20",
    "LUC7L3",
    "SNORD49A",
    "CTSH",
    "ACOT8",
]

  you can then call **querymany** method, telling it your input is "symbol", and you want "entrezgene" (Entrez gene ids) back.

In [ ]:
out = mg.querymany(xli, scopes="symbol", fields="entrezgene", species="human")

Returned "**out**" looks like this:

In [ ]:
out

The mapping result is returned as a list of dictionaries. Each dictionary contains the **fields** you asked to return, in this case, "entrezgene" field. Each dictionary also returns the matching query term, "**query**", and an internal id, "**_id**", which is the same as "**entrezgene**" most of time (will be an ensembl gene id if a gene is available from Ensembl only).

From here we can easily parse **out** to create a mapping from gene symbol to entrez id. Done!

In [ ]:
symbol_to_entrez = dict()
for entry in out:
    symbol_to_entrez[entry['query']] = entry['entrezgene']
    
symbol_to_entrez

---

## Mapping gene symbols to Ensembl gene ids

Now if you want Ensembl gene ids back:

In [ ]:
mg.querymany(xli, scopes="symbol", fields="ensembl.gene", species="human")

## When an input id has no matching gene

In case that an input id has no matching gene, you will be notified from the output.The returned dictionary for this query term contains "notfound" value as *True*.

In [ ]:
xli = ["CCDC83", "MAST3", "FLOT1", "RPL11", "Gm10494"]

In [ ]:
mg.querymany(xli, scopes="symbol", fields="entrezgene", species="human")

## When input ids are not just symbols

In [ ]:
xli = [
    "DDX26B",
    "CCDC83",
    "MAST3",
    "FLOT1",
    "RPL11",
    "Gm10494",
    "1007_s_at",
    "AK125780",
]

Above id list contains symbols, reporters and accession numbers, and supposedly we want to get back both Entrez gene ids and uniprot ids. Parameters **scopes**, **fields**, **species** are all flexible enough to support multiple values, either a list or a comma-separated string:

In [ ]:
mg.querymany(
    xli,
    scopes="symbol,reporter,accession",
    fields="entrezgene,uniprot",
    species="human",
)

## When a input id has multiple matching genes

From the previous result, you may have noticed that query term "1007_s_at" matches two genes. In that case, you will be notified from the output, and the returned result will include both matching genes.

By passing "returnall=True", you will get both duplicate or missing query terms, together with the mapping output, from the returned result:

In [ ]:
mg.querymany(
    xli,
    scopes="symbol,reporter,accession",
    fields="entrezgene,uniprot",
    species="human",
    returnall=True,
)

The returned result above contains "**out**" for mapping output, "**missing**" for missing query terms (a list), and "**dup**" for query terms with multiple matches (including the number of matches).

## Can I convert a very large list of ids?

Yes, you can. If you pass an id list (i.e., **xli** above) larger than 1000 ids, we will do the id mapping in-batch with 1000 ids at a time, and then concatenate the results all together for you. So, from the user-end, it's exactly the same as passing a shorter list. You don't need to worry about saturating our backend servers.

## To read more

* [MyGene.info](http://mygene.info)
    * [Batch queries via POST](http://mygene.info/doc/query_service.html#batch-queries-via-post)
        * [**scopes** parameter](http://mygene.info/doc/query_service.html#scopes)
        * [**fields** parameter](http://mygene.info/doc/query_service.html#fields)
        * [**species** parameter](http://mygene.info/doc/query_service.html#species)
* [mygene module](https://crate.io/packages/mygene/)